In [3]:
import pandas as pd
import numpy as np
import json
from edgar import *
from tqdm import tqdm
from joblib import Parallel, delayed
import pickle
import multiprocessing
set_identity('80f9jiojojioqq.com@gmail.com')

[00:20:37] INFO     Identity of the Edgar REST client set to [80f9jiojojioqq.com@gmail.com]             ]8;id=535650;file:///opt/anaconda3/envs/learn/lib/python3.9/site-packages/edgar/core.py\core.py]8;;\:]8;id=300894;file:///opt/anaconda3/envs/learn/lib/python3.9/site-packages/edgar/core.py#158\158]8;;\

In [4]:
list = [year for year in range(2005, 2010)]
# get 10-K and 10-Q filings for 2024

filings = get_filings(year=list, form=['10-K', '10-Q' ,'DEF 14A'])
print(f' Runing all')


# this is a regex function, it requires at least a verd and a noun exist, and their distance is less than 20 words 
verbs_1 = ['freeze', 'froze', 'frozen', 'freezing', 'close', 'closed', 'discontinue', 'discontinued', 'terminate', 'terminated', 'renegotiate', 'renegotiated']
nouns_1 = ['defined benefit', 'pension plan', 'retirement', 'postretirement', 'postemployment', 'pension', 'benefit']

# Create a regex pattern
pattern_1 = r'\b(?:' + '|'.join(re.escape(word) for word in verbs_1) + r')\b(?:\W+\w+){0,20}?\W+\b(?:' + '|'.join(re.escape(word) for word in nouns_1) + r')\b' + \
            r'|\b(?:' + '|'.join(re.escape(word) for word in nouns_1) + r')\b(?:\W+\w+){0,20}?\W+\b(?:' + '|'.join(re.escape(word) for word in verbs_1) + r')\b'

# for pattern 2
verbs_2 = ['move', 'moved', 'transfer', 'transferred', 'transfering', 'transfered', 'transfering', 'transfered' ,'transit', 'transition','transited','change','changed','turn','turned','new employee']
nouns_2 = ['defined benefit', 'pension plan', 'retirement', 'postretirement', 'defined contribution', 'contribution', '401(k)']

# pattern 2
pattern_2 = r'\b(?:' + '|'.join(re.escape(word) for word in verbs_2) + r')\b(?:\W+\w+){0,20}?\W+\b(?:' + '|'.join(re.escape(word) for word in nouns_2) + r')\b' + \
            r'|\b(?:' + '|'.join(re.escape(word) for word in nouns_2) + r')\b(?:\W+\w+){0,20}?\W+\b(?:' + '|'.join(re.escape(word) for word in verbs_2) + r')\b'

def find_freeze(filing):
    
    try:
        result_1 = filing.search(pattern_1, regex=True).json() # search for keywords
    except:
        result_1 = None   

    try:
        result_2 = filing.search(pattern_2, regex=True).json() # search for keywords
    except:
        result_2 = None     

    return filing, (result_1, result_2)




res = Parallel(n_jobs=-1)(delayed(find_freeze)(filing) for filing in tqdm(filings))


freeze_collection = dict(res)

# save the collection
with open(f'freeze_collection_10.pkl', 'wb') as f:
    pickle.dump(freeze_collection, f)


TooManyRequestsError: Too Many Requests